In [9]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.prompts import PromptTemplate
import json
import requests

api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERP_API_KEY")
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "llmapps"



In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = OpenAI(temperature=0)


In [ ]:
#Load the documents

from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.pdf")
    return dl.load()

documents = find_md_files('Resumes')
len(documents)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

pdf_text_splitter = CharacterTextSplitter(chunk_size=1500,chunk_overlap  = 200)
document_sections = pdf_text_splitter.split_documents(documents)
len(document_sections)


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)


In [ ]:
retriever = db.as_retriever(search_kwargs=dict(k=3))


In [ ]:
query = "The resume of graphic"
docs = retriever.get_relevant_documents(query)


In [ ]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
query = "Tell me more about the graphic desingers"
qa.run(query)
